<a href="https://colab.research.google.com/github/Capstone-Team-CH2-PS379/Machine-Learning/blob/main/model-experiments/Speech_Recognition_Model_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from transformers import Wav2Vec2Processor, TFWav2Vec2ForCTC
from tensorflow.keras import layers
from datasets import load_dataset

# Load dataset
dataset = load_dataset("speech_commands", "v0.02", split="train")


Generating train split:   0%|          | 0/84848 [00:00<?, ? examples/s]

KeyboardInterrupt: ignored

In [ ]:
# Inisialisasi processor dan model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = TFWav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# preprocessing data
def preprocess_data(data):
    audio = data["audio"]
    # Extract the audio array from the dictionary
    audio_array = audio["array"]

    input_values = processor(audio_array, return_tensors="tf", padding="max_length", truncation=True, max_length=32000).input_values
    return {"input_values": input_values}

# Preprocessing label
def preprocess_label(data):
    return {"ctc_labels": tf.keras.backend.ctc_label_dense_to_sparse(data["label"], 0)}

# Praproses dataset
dataset = dataset.map(preprocess_data)
dataset = dataset.map(preprocess_label)

# Dataset to set train & validasi
train_dataset = dataset.shuffle(100).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)



TFWav2Vec2ForCTC has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
All PyTorch model weights were used when initializing TFWav2Vec2ForCTC.

Some weights or buffers of the TF 2.0 model TFWav2Vec2ForCTC were not initialized from the PyTorch model and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/84848 [00:00<?, ? examples/s]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_ra

In [ ]:
input_layer = layers.Input(shape=(None,), dtype=tf.int32, name="input_values")
output = model(input_layer, training=True).logits
ctc_model = tf.keras.models.Model(inputs=input_layer, outputs=output)

def ctc_loss(y_true, y_pred):
    ctc_labels = tf.cast(y_true, tf.int32)
    input_length = tf.ones_like(ctc_labels) * tf.shape(y_pred)[1]
    label_length = tf.ones_like(ctc_labels) * tf.shape(ctc_labels)[1]
    loss = tf.nn.ctc_loss(ctc_labels, y_pred, input_length, label_length)
    return tf.reduce_mean(loss)

# Kompilasi model
ctc_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=ctc_loss)

# Latih model
ctc_model.fit(train_dataset, epochs=5)
